In [ ]:
import os, numpy as np

# geometry & BCs
OUTER_IN = 9.0
INNER_IN = 3.0
ICE_UP_IN = 4.0
T_BATH = 32.0
T_TOP = 100.0
T_STEAM = 212.0

# grid
N = 81
y = np.arange(N)
x = np.arange(N)

def side_temp(j):
    yy = j*(OUTER_IN/(N-1))
    if yy <= ICE_UP_IN: return T_BATH
    t = (yy - ICE_UP_IN)/(OUTER_IN - ICE_UP_IN)
    return (1-t)*T_BATH + t*T_TOP

# inner indices
gap_in = (OUTER_IN - INNER_IN)/2
gap_idx = int(round(gap_in*(N-1)/OUTER_IN))
inner_w_idx = int(round(INNER_IN*(N-1)/OUTER_IN))
ix0 = gap_idx
ix1 = ix0 + inner_w_idx
iy0 = gap_idx
iy1 = iy0 + inner_w_idx

# field + mask
A = np.zeros((N,N),float)
fixed = np.zeros_like(A,bool)

A[0,:]  = T_BATH; fixed[0,:]  = True
A[-1,:] = T_TOP;  fixed[-1,:] = True
for j in range(N):
    v = side_temp(j)
    A[j,0]  = v; fixed[j,0]  = True
    A[j,-1] = v; fixed[j,-1] = True

A[iy0:iy1+1, ix0:ix1+1] = T_STEAM
fixed[iy0:iy1+1, ix0:ix1+1] = True

# symmetry
HALF = N//2 + 1
Ah = A[:, :HALF].copy()
fixed_h = fixed[:, :HALF]

def sweep(Ah, fixed_h):
    md = 0.0
    for j in range(1, N-1):
        for i in range(1, HALF):
            if fixed_h[j,i]:
                continue
            left = Ah[j,i-1]
            right = Ah[j,i+1] if i < HALF-1 else Ah[j,i-1]
            up = Ah[j+1,i]
            down = Ah[j-1,i]
            old = Ah[j,i]
            new = 0.25*(left + right + up + down)
            d = abs(new-old)
            if d > md:
                md = d
            Ah[j,i] = new
    return md

def full_from_half(Ah):
    right = Ah[:,-2::-1]
    return np.concatenate([Ah, right], axis=1)

# VTK writers 
def write_vti(path, A, spacing, origin=(0.0, 0.0, 0.0), name="Temperature"):
    """
    Writes a 2D scalar field as VTK XML ImageData (.vti) with PointData (Z extent = 1).
    """
    nx, ny = A.shape[1], A.shape[0]  # x columns, y rows
    dx, dy, dz = spacing
    ox, oy, oz = origin

    with open(path, "w") as f:
        f.write('<?xml version="1.0"?>\n')
        f.write('<VTKFile type="ImageData" version="0.1" byte_order="LittleEndian">\n')
        f.write(f'  <ImageData WholeExtent="0 {nx-1} 0 {ny-1} 0 0" Origin="{ox} {oy} {oz}" Spacing="{dx} {dy} {dz}">\n')
        f.write(f'    <Piece Extent="0 {nx-1} 0 {ny-1} 0 0">\n')
        f.write(f'      <PointData Scalars="{name}">\n')
        f.write(f'        <DataArray type="Float32" Name="{name}" format="ascii">\n')

        # VTK expects x-fastest, then y, then z. Our A[y, x].
        # z is 0..0 since it's 2D.
        for j in range(ny):
            # write a row (space-separated) for readability
            row_vals = " ".join(f"{float(A[j, i]):.6g}" for i in range(nx))
            f.write("          " + row_vals + "\n")

        f.write('        </DataArray>\n')
        f.write('      </PointData>\n')
        f.write('      <CellData/>\n')
        f.write('    </Piece>\n')
        f.write('  </ImageData>\n')
        f.write('</VTKFile>\n')

def write_pvd(path, datasets):
    """
    datasets: list of tuples (timestep_float, filename_str) in desired order.
    """
    with open(path, "w") as f:
        f.write('<?xml version="1.0"?>\n')
        f.write('<VTKFile type="Collection" version="0.1" byte_order="LittleEndian">\n')
        f.write('  <Collection>\n')
        for ts, fn in datasets:
            f.write(f'    <DataSet timestep="{float(ts)}" group="" part="0" file="{fn}"/>\n')
        f.write('  </Collection>\n')
        f.write('</VTKFile>\n')


os.makedirs("data", exist_ok=True)

TARGETS = [1, 10, 50, 100, 'stable', 200]  # 6 outputs; 5th is the stable one
target_idx = 0

TOL = 0.5
it = 0
stable_it = None
MAX_IT = 20000


dx = dy = OUTER_IN / (N - 1)
dz = 1.0  
origin = (0.0, 0.0, 0.0)

pvd_entries = []

while target_idx < len(TARGETS) and it < MAX_IT:
    it += 1
    md = sweep(Ah, fixed_h)

    # detect stability once
    if stable_it is None and md <= TOL:
        stable_it = it

    target = TARGETS[target_idx]

    # add checkpoints
    
    if isinstance(target, int) and it == target:
        Afull = full_from_half(Ah)
        vti_name = f"step_{it:04d}.vti"
        write_vti(os.path.join("data", vti_name), Afull, spacing=(dx, dy, dz), origin=origin, name="Temperature")
        pvd_entries.append((it, vti_name))
        target_idx += 1
        continue

    if target == 'stable' and stable_it is not None:
        Afull = full_from_half(Ah)
        vti_name = f"step_{stable_it:04d}Stable.vti"
        write_vti(os.path.join("data", vti_name), Afull, spacing=(dx, dy, dz), origin=origin, name="Temperature")
        pvd_entries.append((stable_it, vti_name))
        target_idx += 1
        continue

# PVD conversion
write_pvd(os.path.join("data", "series.pvd"), pvd_entries)
